In [6]:
import pandas as pd
import os
import re
from nltk.corpus import stopwords
import numpy as np
import time
from stemming.porter2 import stem

In [4]:
def load(dirname):
    df = pd.DataFrame(columns = ['text', 'summary', 'target', 'entities'])
    for filename in os.listdir(dirname):
        if filename.endswith('.summary'):
            pathname = os.path.join(dirname, filename)
            textfile = open(pathname, encoding='utf8')
            contents = textfile.read()
            entries = []
            for entry in contents.split('\n\n'):
                entries.append(entry)
            a = entries[1]
            a = a.split('\n')
            target = []
            text = ''
            for i in range(len(a)):
                text += str(a[i].split('\t\t\t')[0])+'\n\n'
                target.append(a[i].split('\t\t\t')[1])
            dict1 = {}
            for i in range(len(entries[3].split('\n'))):
                dict1[str(entries[3].split('\n')[i].split(':')[0])] = str(entries[3].split('\n')[i].split(':')[1])
            sentences = text.split('\n\n')
            sent = entries[2].split('\n\n')
            for i in range(len(sentences)):
                tokens = sentences[i].split()
                for j in range(len(tokens)):
                    if tokens[j] in dict1:
                        tokens[j] = dict1[tokens[j]]
                sentences[i] = " ".join(tokens)
            for i in range(len(sent)):
                token = sent[i].split()
                for j in range(len(token)):
                    if token[j] in dict1:
                        token[j] = dict1[token[j]]
                sent[i] = " ".join(token)
            text = "\n\n".join(sentences)
            summary = "\n\n".join(sent)
            l = [str(text), str(summary), target, dict1]
            df2 = pd.DataFrame([l], columns=['text', 'summary', 'target', 'entities'])
            df = df.append(df2, ignore_index=True)
    return(df)

In [5]:
df = load('/dataset/training')

NameError: name 'pd' is not defined

In [6]:
documents = []
for i in range(len(df["text"])):
    a = df["text"][i].split('\n\n')
    documents.append(a[:-1])

In [7]:
targets = []
for i in range(len(df["target"])):
    targets.append(df["target"][i])

In [7]:
contractions = { 
"ain't": "am not",
"aren't": "are not",
"can't": "cannot",
"can't've": "cannot have",
"'cause": "because",
"could've": "could have",
"couldn't": "could not",
"couldn't've": "could not have",
"didn't": "did not",
"doesn't": "does not",
"don't": "do not",
"hadn't": "had not",
"hadn't've": "had not have",
"hasn't": "has not",
"haven't": "have not",
"he'd": "he would",
"he'd've": "he would have",
"he'll": "he will",
"he's": "he is",
"how'd": "how did",
"how'll": "how will",
"how's": "how is",
"i'd": "i would",
"i'll": "i will",
"i'm": "i am",
"i've": "i have",
"isn't": "is not",
"it'd": "it would",
"it'll": "it will",
"it's": "it is",
"let's": "let us",
"ma'am": "madam",
"mayn't": "may not",
"might've": "might have",
"mightn't": "might not",
"must've": "must have",
"mustn't": "must not",
"needn't": "need not",
"oughtn't": "ought not",
"shan't": "shall not",
"sha'n't": "shall not",
"she'd": "she would",
"she'll": "she will",
"she's": "she is",
"should've": "should have",
"shouldn't": "should not",
"that'd": "that would",
"that's": "that is",
"there'd": "there had",
"there's": "there is",
"they'd": "they would",
"they'll": "they will",
"they're": "they are",
"they've": "they have",
"wasn't": "was not",
"we'd": "we would",
"we'll": "we will",
"we're": "we are",
"we've": "we have",
"weren't": "were not",
"what'll": "what will",
"what're": "what are",
"what's": "what is",
"what've": "what have",
"where'd": "where did",
"where's": "where is",
"who'll": "who will",
"who's": "who is",
"won't": "will not",
"wouldn't": "would not",
"you'd": "you would",
"you'll": "you will",
"you're": "you are"
}

In [8]:
def preprocess(text, stopwords_removal=False, stemming=False):
    cachedStopWords = stopwords.words("english")
    text = text.lower()
    text = re.sub(r'[_"\-;%()|+&=*%.,!?:#$@\[\]/]', ' ', text)
    text = re.sub(r" ' ", " ", text)
    str1 = re.findall("[@a-zA-Z0-9]* '[a-zA-Z]", text)
    for l in str1:
        text= re.sub(l,l.replace(" ",""),text)
    text = re.findall(r"[\w']+", text)
    new_text = []
    for word in text:
        if word in contractions:
            new_text.append(contractions[word])
        else:
            new_text.append(word)
    text = " ".join(new_text)
    tokens = text.split()
    words = []
    if stopwords_removal:
        for token in tokens:
            if token not in cachedStopWords:
                words.append(token)
            text = " ".join(words)        
    words = []
    if stemming:
        for token in tokens:
            words.append(stem(token))
        text = " ".join(words)
    return text

In [10]:
for i in range(len(documents)):
    for j in range(len(documents[i])):
        documents[i][j] = preprocess(documents[i][j], True)

In [11]:
__pickleStuff("documents.p",documents)
__pickleStuff("targets.p",targets)

In [2]:
documents = __loadStuff("documents.p")
targets = __loadStuff("targets.p")


In [5]:
embeddings_index = {}
with open('/numberbatch/numberbatch.txt', encoding='utf-8') as f:
    for line in f:
        values = line.split(' ')
        word = values[0]
        embedding = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = embedding

print('Word embeddings:', len(embeddings_index))

Word embeddings: 1917248


In [6]:
def count_words(count_dict, documents):
    for i in range(len(documents)):
        for sentence in documents[i]:
            for word in sentence.split():
                if word not in count_dict:
                    count_dict[word] = 1
                else:
                    count_dict[word] += 1 

In [7]:
word_counts = {}

count_words(word_counts, documents)

print("Size of vocabulary :", len(word_counts))

Size of vocabulary : 416253


In [8]:
missing_words = 0
threshold = 15

for word, count in word_counts.items():
    if count>threshold:
        if word not in embeddings_index:
            missing_words += 1
missing_ratio = round(missing_words/len(word_counts), 4)*100

print("Number of words missing from CN:", missing_words)
print("Percent of words that are missing from vocabulary: {}%".format(missing_ratio))

Number of words missing from CN: 85188
Percent of words that are missing from vocabulary: 20.47%


In [9]:
missing_words = []
for word, count in word_counts.items():
    if count > threshold and word not in embeddings_index:
        missing_words.append((word,count))
missing_words[:30]

[('metodiev', 25),
 ('portfolios', 65),
 ('jiabao', 71),
 ('mh17', 1192),
 ("huguely's", 26),
 ('router', 89),
 ('verges', 103),
 ('thermomix', 158),
 ('cuban', 1153),
 ('laxey', 31),
 ('frowning', 51),
 ('kissed', 1210),
 ('fortifications', 62),
 ('attire', 746),
 ('airspeed', 36),
 ('reuters', 2012),
 ('themed', 1840),
 ('celta', 259),
 ('triggers', 524),
 ('liquidator', 19),
 ('padang', 28),
 ('ketosis', 21),
 ('islamophobic', 64),
 ('blindness', 466),
 ('meter', 803),
 ('liege', 189),
 ('unamused', 27),
 ('daze', 84),
 ('houndstooth', 22),
 ('kerosene', 105)]

In [10]:
#dictionary to convert words to integers
vocab_to_int = {} 
# Index words from 0
value = 0
for word, count in word_counts.items():
    if count >= threshold or word in embeddings_index:
        vocab_to_int[word] = value
        value += 1

# Special tokens that will be added to our vocab
codes = ["<UNK>","<PAD>","<EOS>","<GO>"]   

# Add codes to vocab
for code in codes:
    vocab_to_int[code] = len(vocab_to_int)

# Dictionary to convert integers to words
int_to_vocab = {}
for word, value in vocab_to_int.items():
    int_to_vocab[value] = word

usage_ratio = round(len(vocab_to_int) / len(word_counts),4)*100

print("Total number of unique words:", len(word_counts))
print("Number of words we will use:", len(vocab_to_int))
print("Percent of words we will use: {}%".format(usage_ratio))

Total number of unique words: 416253
Number of words we will use: 88037
Percent of words we will use: 21.15%


In [11]:
# Need to use 300 for embedding dimensions to match CN's vectors.
embedding_dim = 300
nb_words = len(vocab_to_int)

# Create matrix with default values of zero
word_embedding_matrix = np.zeros((nb_words, embedding_dim), dtype=np.float32)
for word, i in vocab_to_int.items():
    if word in embeddings_index:
        word_embedding_matrix[i] = embeddings_index[word]
    else:
        # If word not in CN, create a random embedding for it
        new_embedding = np.array(np.random.uniform(-1.0, 1.0, embedding_dim))
        embeddings_index[word] = new_embedding
        word_embedding_matrix[i] = new_embedding

# Check if value matches len(vocab_to_int)
print(len(word_embedding_matrix))

88037


In [12]:
def convert_to_ints(docs, word_count, unk_count, eos=False):
    '''Convert words in text to an integer.
       If word is not in vocab_to_int, use UNK's integer.
       Total the number of words and UNKs.
       Add EOS token to the end of texts'''
    ints = []
    for text in docs:    
        for sentence in text:
            sentence_ints = []
            for word in sentence.split():
                word_count += 1
                if word in vocab_to_int:
                    sentence_ints.append(vocab_to_int[word])
                else:
                    sentence_ints.append(vocab_to_int["<UNK>"])
                    unk_count += 1
            if eos:
                sentence_ints.append(vocab_to_int["<EOS>"])
            ints.append(sentence_ints)
    return ints, word_count, unk_count

In [13]:
word_count = 0
unk_count = 0

In [14]:
int_texts, word_count, unk_count = convert_to_ints(documents, word_count, unk_count, eos=True)

unk_percent = round(unk_count/word_count,4)*100

print("Total number of words in documents:", word_count)
print("Total number of UNKs in documents:", unk_count)
print("Percent of words that are UNK: {}%".format(unk_percent))

Total number of words in documents: 76674995
Total number of UNKs in documents: 999119
Percent of words that are UNK: 1.3%


In [15]:
len(int_texts[:3][0])

9

In [16]:
def create_lengths(text):
    '''Create a data frame of the sentence lengths from a text'''
    lengths = []
    for sentence in text:
        lengths.append(len(sentence))
    return pd.DataFrame(lengths, columns=['counts'])

In [18]:
text_lengths= create_lengths(int_texts)

In [19]:
print(np.percentile(text_lengths.counts, 90))
print(np.percentile(text_lengths.counts, 95))
print(np.percentile(text_lengths.counts, 99))

28.0
35.0
56.0


In [1]:
import pickle
def __pickleStuff(filename, stuff):
    save_stuff = open(filename, "wb")
    pickle.dump(stuff, save_stuff)
    save_stuff.close()
def __loadStuff(filename):
    saved_stuff = open(filename,"rb")
    stuff = pickle.load(saved_stuff)
    saved_stuff.close()
    return stuff

# __pickleStuff("documents.p",documents)
# __pickleStuff("targets.p",targets)

# __pickleStuff("int_texts.p",int_texts)
# __pickleStuff("text_lenghts.p",text_lengths)
# __pickleStuff("word_embedding_matrix.p",word_embedding_matrix)

# __pickleStuff("vocab_to_int.p",vocab_to_int)
# __pickleStuff("int_to_vocab.p",int_to_vocab)

In [2]:
# documents = __loadStuff("/dataset/documents.p")
targets = __loadStuff("targets.p")

# int_texts = __loadStuff("/dataset/int_texts.p")
# text_lengths = __loadStuff("/dataset/text_lenghts.p")
# word_embedding_matrix = __loadStuff("/dataset/word_embedding_matrix.p")

# vocab_to_int = __loadStuff("/dataset/vocab_to_int.p")
# int_to_vocab = __loadStuff("/dataset/int_to_vocab.p")

## Model

In [8]:
import tensorflow as tf

In [9]:
def model_inputs():
    inputs_X = tf.placeholder(tf.int32, [None, None], name='inputs_X')
    inputs_Y = tf.placeholder(tf.float32, [None, None], name='inputs_Y')
    lr = tf.placeholder(tf.float32, name='learning_rate')
    keep_prob = tf.placeholder(tf.float32, name='keep_prob')
    seq_length = tf.placeholder(tf.int32, (None,), name='seq_length')
    max_seq_length = tf.reduce_max(seq_length, name='max_seq_length')
    
    return inputs_X, inputs_Y, lr, seq_length, max_seq_length, keep_prob


def process_rnn_input(target_data, vocab_to_int, batch_size):  
    ending = tf.strided_slice(target_data, [0, 0], [batch_size, -1], [1, 1]) # slice it to target_data[0:batch_size, 0: -1]
    dec_input = tf.concat([tf.fill([batch_size, 1], vocab_to_int['<GO>']), ending], 1)

    return dec_input


def unirnn_layer(rnn_size, sequence_length, num_layers, rnn_inputs, keep_prob):
    for layer in range(num_layers):
        with tf.variable_scope("rnn_{}".format(layer)):
            single_cell = tf.contrib.rnn.LSTMCell(rnn_size)
            single_cell = tf.contrib.rnn.DropoutWrapper(single_cell, input_keep_prob = keep_prob)


            enc_output, enc_state = tf.nn.dynamic_rnn(single_cell, rnn_inputs, sequence_length, dtype=tf.float32)
    
    return enc_output, enc_state


def birnn_layer(rnn_size, sequence_length, num_layers, rnn_inputs, keep_prob):
    for layer in range(num_layers):
        with tf.variable_scope("birnn_{}".format(layer)):
            cell_fw = tf.contrib.rnn.LSTMCell(rnn_size, initializer=tf.random_uniform_initializer(-0.1, 0.1, seed=2))
            cell_fw = tf.contrib.rnn.DropoutWrapper(cell_fw, input_keep_prob = keep_prob)

            cell_bw = tf.contrib.rnn.LSTMCell(rnn_size, initializer=tf.random_uniform_initializer(-0.1, 0.1, seed=2))
            cell_bw = tf.contrib.rnn.DropoutWrapper(cell_bw, input_keep_prob = keep_prob)
            enc_output, enc_state = tf.nn.bidirectional_dynamic_rnn(cell_fw, cell_bw, rnn_inputs, sequence_length, dtype=tf.float32)
            
            enc_output = tf.concat(enc_output, 2)
            enc_state = tf.concat(enc_state, 2)
            
    return enc_output, enc_state



def sequence_classifier(input_data, target_data, num_classes, keep_prob, text_length, max_seq_length, 
                  vocab_size, rnn_size, num_layers, vocab_to_int, batch_size):
    embeddings = word_embedding_matrix
    enc_embed_input = tf.nn.embedding_lookup(embeddings, input_data)
    print(enc_embed_input.shape)
    all_outputs, state = unirnn_layer(rnn_size, text_length, num_layers, enc_embed_input, keep_prob)
#     print(state.get_shape())
    outputs = state[0]
            
#     logits_rnn = rnn_softmax(outputs)
    with tf.variable_scope("rnn_softmax"):
        W_softmax = tf.get_variable("W_softmax", [300, num_classes])
        b_softmax = tf.get_variable("b_softmax", [num_classes])
    logits_rnn = tf.matmul(outputs, W_softmax) + b_softmax
    
    return logits_rnn



def pad_sentence_batch(sentence_batch):
    """Pad sentences with <PAD> so that each sentence of a batch has the same length"""
    max_sentence = max([len(sentence) for sentence in sentence_batch])
    return [sentence + [vocab_to_int['<PAD>']] * (max_sentence - len(sentence)) for sentence in sentence_batch]


def get_batches(target_data, texts, batch_size):
    """Batch targets, texts, and the lengths of their sentences together"""
    for batch_i in range(0, len(texts)//batch_size):
        start_i = batch_i * batch_size
        targets_batch = target_data[start_i:start_i + batch_size]
        texts_batch = texts[start_i:start_i + batch_size]
        pad_texts_batch = np.array(pad_sentence_batch(texts_batch), dtype=np.float)
#         targets_batches = np.array(targets_batch)
#         print(targets_batch.shape)
        
        pad_texts_lengths = []
        for text in pad_texts_batch:
            pad_texts_lengths.append(len(text))
        
        pad_texts_lengths = np.array(pad_texts_lengths, dtype=np.float)
        
        yield targets_batch, pad_texts_batch, pad_texts_lengths
        
        


In [110]:
targets_y = []

for i in range(len(targets)):
    for j in range(len(targets[i])):
        targets_y.append(targets[i][j])

for i in range(len(targets_y)):
    if targets_y[i] == str(2):
        targets_y[i] = "0"
        
targets_y = np.asarray(targets_y, dtype=np.float)
targets_y = targets_y.reshape(-1, 1)

In [111]:
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder()
targetsenc = enc.fit_transform(targets_y)

In [25]:
targetsenc = targetsenc.toarray()
int_texts = np.asarray(int_texts)

In [26]:
print("'<PAD>' has id: {}".format(vocab_to_int['<PAD>']))
targets_samples = targets_y[7:50]
texts_samples = int_texts[7:50]
targets_batch_samples, pad_texts_batch_samples, pad_texts_lengths_samples = next(get_batches(
    targets_samples, texts_samples, 5))
print("pad texts batch samples:\n\r {}".format(pad_texts_batch_samples))

'<PAD>' has id: 88034
pad texts batch samples:
 [[ 65211.  88035.  88034.  88034.  88034.  88034.  88034.  88034.  88034.
   88034.  88034.  88034.  88034.  88034.  88034.]
 [ 10174.  88033.  15435.  88035.  88034.  88034.  88034.  88034.  88034.
   88034.  88034.  88034.  88034.  88034.  88034.]
 [ 31686.  88033.  22055.  25918.  71160.  14639.  84793.   1036.  29593.
   42605.  26617.  77459.  18333.  25278.  88035.]
 [ 31686.  28681.  50739.  67122.  66741.  19572.  12396.  88035.  88034.
   88034.  88034.  88034.  88034.  88034.  88034.]
 [ 31686.  69477.  37647.  41305.  74368.  22722.  46160.  88035.  88034.
   88034.  88034.  88034.  88034.  88034.  88034.]]


In [10]:
# Set the Hyperparameters
epochs = 100
batch_size = 128
rnn_size = 300
num_layers = 3
num_classes = 3
learning_rate = 0.0025
keep_probability = 0.95
max_gradient_norm = 5.0

In [28]:
train_graph = tf.Graph()
with train_graph.as_default():
    
    inputs_X, inputs_Y, lr, seq_length, max_seq_length, keep_prob = model_inputs()
    
    logs = sequence_classifier(inputs_X, inputs_Y, num_classes, keep_prob, seq_length, max_seq_length, 
                  len(vocab_to_int)+1, rnn_size, num_layers, vocab_to_int, batch_size)
#     print(logs.shape)
    predictions = tf.nn.softmax(logs)
    predictions = tf.identity(predictions, name='predictions')
#     print(predictions.shape)
    accuracy = tf.equal(tf.argmax(inputs_Y,1), tf.argmax(predictions,1))
    
    loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=logs, labels=inputs_Y))
    
    with tf.name_scope("optimization"):
        trainable_vars = tf.trainable_variables()
        
        grads, _ = tf.clip_by_global_norm(tf.gradients(loss, trainable_vars), max_gradient_norm)
        
        optimizer = tf.train.AdamOptimizer(learning_rate)
        train_optimizer = optimizer.apply_gradients(zip(grads, trainable_vars))

print("Graph is built.")
graph_location = "./graph"
print(graph_location)
train_writer = tf.summary.FileWriter(graph_location)
train_writer.add_graph(train_graph)

(?, ?, 300)
Graph is built.
./graph


In [29]:
max_seq = 56

In [30]:
# Train the Model
learning_rate_decay = 0.55
min_learning_rate = 0.0005
display_step = 20 # Check training loss after every 20 batches
stop_early = 0 
stop = 7 # If the update loss does not decrease in 3 consecutive update checks, stop training
per_epoch = 3 # Make 3 update checks per epoch
update_check = (len(int_texts)//batch_size//per_epoch)-1

update_loss = 0 
batch_loss = 0
classifier_update_loss = [] # Record the update losses for saving improvements in the model
classifier_accuracy = []
predict = []

checkpoint = "./best_model_classifier.ckpt" 
with tf.Session(graph=train_graph) as sess:
    sess.run(tf.global_variables_initializer())
    
    # If we want to continue training a previous session
    #loader = tf.train.import_meta_graph("./" + checkpoint + '.meta')
    #loader.restore(sess, checkpoint)
    
    for epoch_i in range(1, epochs+1):
        update_loss = 0
        batch_loss = 0
        for batch_i, (targets_batch, pad_texts_batch, pad_texts_lengths) in enumerate(
                get_batches(targetsenc, int_texts, batch_size)):
            start_time = time.time()
#             print(type(pad_texts_batch), type(targets_batch), type(pad_texts_lengths))
            _, losss, accuracyy, logets = sess.run(
                [train_optimizer, loss, accuracy, predictions],
                {inputs_X: pad_texts_batch,
                 inputs_Y: list(targets_batch),
                 lr: learning_rate,
                 seq_length: list(pad_texts_lengths),
                 keep_prob: keep_probability})

            batch_loss += losss
            update_loss += losss
            end_time = time.time()
            batch_time = end_time - start_time
            predict.append(logets)
            
            if batch_i % display_step == 0 and batch_i > 0:
                print('Epoch {:>3}/{} Batch {:>4}/{} - Loss: {:>6.3f}, Seconds: {:>4.2f}'
                      .format(epoch_i,
                              epochs, 
                              batch_i, 
                              len(int_texts) // batch_size, 
                              batch_loss / display_step, 
                              batch_time*display_step))
                print("Accuracy is: ", np.mean(accuracyy))
                classifier_accuracy.append(accuracyy)
                batch_loss = 0

            if batch_i % update_check == 0 and batch_i > 0:
                print("Average loss for this update:", round(update_loss/update_check,3))
                classifier_update_loss.append(update_loss)
                print("Average accuracy for this update %.3f" % np.mean(classifier_accuracy))
                
                # If the update loss is at a new minimum, save the model
                if update_loss <= min(classifier_update_loss):
                    print('New Record!') 
                    stop_early = 0
                    saver = tf.train.Saver() 
                    saver.save(sess, checkpoint)

                else:
                    print("No Improvement.")
                    stop_early += 1
                    if stop_early == stop:
                        break
                update_loss = 0
            
                    
        # Reduce learning rate, but not below its minimum value
        learning_rate *= learning_rate_decay
        if learning_rate < min_learning_rate:
            learning_rate = min_learning_rate
        
        if stop_early == stop:
            print("Stopping Training.")
            break

Epoch   1/100 Batch   20/39584 - Loss:  1.082, Seconds: 15.85
Accuracy is:  0.3828125
Epoch   1/100 Batch   40/39584 - Loss:  0.615, Seconds: 16.18
Accuracy is:  0.3828125
Epoch   1/100 Batch   60/39584 - Loss:  0.614, Seconds: 13.06
Accuracy is:  0.421875
Epoch   1/100 Batch   80/39584 - Loss:  0.612, Seconds: 23.94
Accuracy is:  0.40625
Epoch   1/100 Batch  100/39584 - Loss:  0.612, Seconds: 14.42
Accuracy is:  0.4609375
Epoch   1/100 Batch  120/39584 - Loss:  0.611, Seconds: 18.50
Accuracy is:  0.40625
Epoch   1/100 Batch  140/39584 - Loss:  0.613, Seconds: 20.18
Accuracy is:  0.4140625
Epoch   1/100 Batch  160/39584 - Loss:  0.613, Seconds: 19.52
Accuracy is:  0.4296875
Epoch   1/100 Batch  180/39584 - Loss:  0.612, Seconds: 26.45
Accuracy is:  0.4375
Epoch   1/100 Batch  200/39584 - Loss:  0.611, Seconds: 22.02
Accuracy is:  0.4296875
Epoch   1/100 Batch  220/39584 - Loss:  0.612, Seconds: 21.53
Accuracy is:  0.421875
Epoch   1/100 Batch  240/39584 - Loss:  0.610, Seconds: 17.56
A

KeyboardInterrupt: 

## Model Evaluation

In [9]:
test = load(r'C:\Users\amrit\OneDrive\Desktop\Thesis\Data\Dataset\cnnanddailymail\test')
test_docs = []

for i in range(len(test["text"])):
    a = test["text"][i].split('\n\n')
    test_docs.append(a[:-1])

test_targets = []
for i in range(len(test["target"])):
    test_targets.append(test["target"][i])
    
test_summaries = []
for i in range(len(test["target"])):
    test_summaries.append(test["summary"][i])
    
# def prep_text(sentence):
#     sentence = preprocess(sentence)
#     return [vocab_to_int.get(word, vocab_to_int['<UNK>']) for word in sentence.split()]

# texts = []

# for document in test_docs:
#     for sentence in document:
#         texts.append(prep_text(sentence))

# len(texts)

In [10]:
"hi"

'hi'

In [58]:
# texts = __loadStuff("test_texts.p")
test_docs = __loadStuff("test_docs.p")
test_targets = __loadStuff("test_targets.p")
for i in range(len(test_docs)):
    for j in range(len(test_docs[i])):
        test_docs[i][j] = preprocess(test_docs[i][j], False, True)

In [13]:
checkpoint = "/checkpoint/./best_model_classifier.ckpt"
loaded_graph = tf.Graph()
answer_logits = []
with tf.Session(graph=loaded_graph) as sess:
    # Load saved model
    loader = tf.train.import_meta_graph(checkpoint + '.meta')
    loader.restore(sess, checkpoint)
    inputs_X = loaded_graph.get_tensor_by_name('inputs_X:0')
    predictions = loaded_graph.get_tensor_by_name('predictions:0')
    seq_length = loaded_graph.get_tensor_by_name('seq_length:0')
    keep_prob = loaded_graph.get_tensor_by_name('keep_prob:0')
    for i, text in enumerate(texts):
        answers = sess.run(predictions, {inputs_X: [text]*batch_size,
                 seq_length: [len(text)]*batch_size,
                 keep_prob: 1.0})
        answer_logits.append(answers)

INFO:tensorflow:Restoring parameters from /checkpoint/./best_model_classifier.ckpt


In [11]:
answer_logits = __loadStuff("answer_logits.p")

In [12]:
answers = []
for i in range(len(answer_logits)):
    answers.append(answer_logits[i][0])

In [13]:
answers = np.asarray(answers)

In [14]:
answers.shape

(264305, 2)

In [15]:
yo = np.argmax(answers, 1)

In [16]:
from collections import Counter
Counter(list(yo))

Counter({0: 34308, 1: 229997})

In [17]:
test_docs_summaries = [[]]
test_output = pd.DataFrame(columns = ['document', 'targets', 'summary', 'predicted_summary'])

test_output["document"] = test_docs
test_output["targets"] = test_targets
test_output["summary"] = test_summaries

# for i in range(len(test_output["document"])):
#     temp = []
#     for j in range(len(test_output["document"][i])):
#         if int(test_output["targets"][i][j]) == 1:
#             temp.append(test_docs[i][j])
#     test_output["predicted_summary"][i] = temp

count = 0
counts = 0
for i in range(len(test_output["document"])):
    temp = []
    for j in range(len(test_output["document"][i])):
        if int(yo[count-1]) == 1 and answers[count-1][1] > 0.99:
            temp.append(test_output["document"][i][j])
        count += 1
    test_output["predicted_summary"][i] = temp

# c = 0

# for i in range(len(test_output["document"])):
#     temp = []
#     index = list(range(c, c+len(test_output["document"][i])))
#     try:
#         index = np.sort(np.argpartition(answers[index][:, 1], -4)[-4:])
#     except:
#         index = np.sort(index)
#     for j in range(len(test_output["document"][i])):
#         if j in index:
#             temp.append(test_output["document"][i][j])
#     test_output["predicted_summary"][i] = temp
#     c += len(test_output["document"][i])
    
    
for i in range(len(test_output["predicted_summary"])):
    test_output["predicted_summary"][i] = ". ".join(test_output["predicted_summary"][i])
# #     test_output["summary"][i] = ". ".join(test_output["summary"][i])

__pickleStuff("test_output_more.p",test_output)

# for i in range(len(test_output["document"])):
#     test_output["document"][i] = ". ".join(test_output["document"][i])

In [18]:
hyps, refs = map(list, zip(*[[test_output["predicted_summary"][i], test_output["summary"][i]] for i in range(len(test_output))]))

In [19]:
hyps[1000], refs[1000]

('Emma Hannigan had her breasts and ovaries removed in 2006 to reduce her risk of cancer after she was diagnosed with the faulty BRCA1 gene a mother - of - two has battled cancer nine times in the last seven years , despite having her breasts and ovaries removed to try and prevent the disease. Emma Hannigan was told in 2005 that she carried the faulty BRCA1 gene , which increases a woman \'s chance of getting breast cancer by 85 per cent , and ovarian cancer by 50 per cent. she opted to have a double mastectomy and oophorectomy the following year , to reduce her risk. since that point , mrs Emma Hannigan has battled the disease a staggering nine times , including four bouts in the space of a year. she is now undergoing chemotherapy every three weeks as specialists try to keep the disease at bay. mrs Emma Hannigan , an author from Bray , near Dublin , said despite it failing to prevent cancer , she does not regret having preventative surgery. the Breast Cancer Campaign estimates prevent

In [17]:
from rouge import Rouge 

rouge = Rouge()
scores = rouge.get_scores(hyps, refs, avg=True)

In [18]:
scores

{'rouge-1': {'f': 0.3166465820724625,
  'p': 0.23719313818615717,
  'r': 0.5189164351979545},
 'rouge-2': {'f': 0.10851889169484978,
  'p': 0.07778537811667763,
  'r': 0.20453484693224744},
 'rouge-l': {'f': 0.13129653532644522,
  'p': 0.12235640579384106,
  'r': 0.337026191124659}}

In [210]:
test_output.head()

,document,targets,summary,predicted_summary
0,[Neil Redfearn is considering his position as ...,"[1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 2, 0, 0, 0, ...",Steve Thompson suspended by Leeds just *19* *g...,Leeds took two months to appoint Steve Thompso...
1,"[she 's a best - selling singer , actress , fa...","[2, 1, 1, 2, 1, 2, 1, 0, 0, 1, 0, 0, 0, 0]","Rita , 24 , models in colourful new Rimmel bea...",Rita has been unveiled as the face of Rimmel '...
2,[the handsome Italy maths teacher who has take...,"[1, 1, 1, 0, 1, 0, 1, 1, 2, 0, 1, 1, 0, 0, 0, ...","Pietro Boselli , 26 , from Brescia in Italy ta...",the handsome Italy maths teacher who has taken...
3,[former Manchester United and Manchester City ...,"[1, 2, 2, 1, 2, 0, 1, 0, 0, 0, 0, 1]","Tevez played for Manchester United , Mancheste...","the Argentinian , who has scored 36 goals in t..."
4,[it is a conversation that will be familiar to...,"[2, 0, 1, 1, 1, 1, 0, 1, 0, 2, 2, 2, 0, 2, 0, ...",Nick Clegg made the *admission* in a rare join...,in a joint interview with his high flying lawy...


In [211]:
__pickleStuff("test_output.p",test_output)

In [1]:
import pickle
def __loadStuff(filename):
    saved_stuff = open(filename,"rb")
    stuff = pickle.load(saved_stuff)
    saved_stuff.close()
    return stuff

In [2]:
test_output = __loadStuff("test_output.p")

In [13]:
". ".join(test_output.document[7111])

"an ex-wife of a North Carolina man serving life in prison for the murder of his third wife has opened up about the abuse she faced when she was married to the man. Casey , also of North Carolina , was married to Michael Wilkie for four years and had a daughter with him before the couple divorced. he went on to marry his third wife , Shelby Wilkie. Michael Wilkie was found guilty of first - degree murder in january for the 2012 killing of Shelby Wilkie and is serving a life sentence without parole. Casey has opened up about the abuse she faced at the hands of Michael Wilkie , who in january was found guilty of the 2012 murder of his third wife ' he said if i ever tried to take his daughter away from him that he would kill me , ' Casey told ABC 's 20/20. Casey and Shelby Wilkie had met Michael Wilkie through an online dating site. Casey said they dated for a year - and - a - half before getting married. ' he was very friendly , very charming , easy to talk to , very soft spoken , and he

In [14]:
test_output.predicted_summary[7111]

"an ex-wife of a North Carolina man serving life in prison for the murder of his third wife has opened up about the abuse she faced when she was married to the man. ' he was very friendly , very charming , easy to talk to , very soft spoken , and he had a good job and seemed to be pretty good , ' Casey said. you would meet when you got home. ' she said , ' i just want to ask you some things about Michael Wilkie , is that ok ? ' and i said , ' sure , ' Casey recalled"

In [15]:
test_output.summary[7111]

'Michael Wilkie was found guilty in january of first - degree murder in january for the 2012 killing of his third wife , Shelby Wilkie his second wife , Casey , has opened up about the abuse she faced before divorcing Casey said that he controlled aspects of her life and was physically abusive , particularly when she was pregnant she even said she feared that Michael Wilkie would kill her she said she never warned Shelby Wilkie , but told her she was there if she needed someone to talk to shortly before she *disappeared*'

In [212]:
test_output = __loadStuff("abs_output.p")

In [219]:
test_output.columns

Index(['document', 'summary', 'predicted_summary'], dtype='object')

In [267]:
for i in range(len(test_output.summary)):
    for j in range(len(test_output["document"][i])):
        if "ronaldo" in test_output["document"][i][j].split(" "):
            print(i, j)

126 24
291 10
2414 8
3857 0
4624 3
8417 29
8417 33
8417 42
8417 55
8587 21
9488 11


In [216]:
"big" in "how big egg tried to bring down little mayo".split(" ")

True

In [19]:
test_output["document"][9488]

['Ronaldo left the pitch with a supporter trying to hug him , Bale left it knowing that once again the fingers of blame will be pointing at him after his first half miss after Diego Godin ’s slip',
 'on a night when Atletico ’s defence once again showed why it is one of the toughest to break down in Europe all three Real forwards were unable to get the better of their rivals',
 'Bale was often the brightest of Real ’s front three but his miss was still the clearest',
 'Bale will not find too much consolation in the fact that he linked well with Ronaldo in the first half',
 'with Atletico ’s left - back position still bearing a Filipe Luis - shaped hole , Bale attacked at will and he looked for Ronaldo with his crosses as the Portuguese found himself in centre - forward territory regularly throughout the first 45 minutes',
 'Ronaldo is confronted by a supporter at the end of the 0 - 0 draw on tuesday night Ronaldo gives the fan a hug after a frustrating night at the Vicente Calderon for

In [20]:
test_output["predicted_summary"][9488]

'Ronaldo left the pitch with a supporter trying to hug him , Bale left it knowing that once again the fingers of blame will be pointing at him after his first half miss after Diego Godin ’s slip. on a night when Atletico ’s defence once again showed why it is one of the toughest to break down in Europe all three Real forwards were unable to get the better of their rivals. with Atletico ’s left - back position still bearing a Filipe Luis - shaped hole , Bale attacked at will and he looked for Ronaldo with his crosses as the Portuguese found himself in centre - forward territory regularly throughout the first 45 minutes. Bale misses the best chance of the match early in the first half as Jan Oblak saves Ronaldo is squeezed out by Atletico midfielders Mario Suarez ( left ) and Gabi when free - kicks came Real ’s way they were almost all taken by Ronaldo , although Bale was allowed to run over one – and in fairness to ronaldo all the set - pieces were awarded on Ronaldo ’s favoured left - 

In [21]:
test_output["summary"][9488]

'Real drew 0 - 0 at Atletico in Champions League quarter - final first *leg* superstars Ronaldo and Bale both started for Real Ronaldo was kept quiet and Bale , though *impressive* , missed best chance'

In [263]:
test_docs_summaries = [[]]
pers_output = pd.DataFrame(columns = ['document', 'targets', 'summary', 'predicted_summary'])

pers_output["document"] = test_docs
pers_output["targets"] = test_targets
pers_output["summary"] = test_summaries

# for i in range(len(test_output["document"])):
#     temp = []
#     for j in range(len(test_output["document"][i])):
#         if int(test_output["targets"][i][j]) == 1:
#             temp.append(test_docs[i][j])
#     test_output["predicted_summary"][i] = temp

count = 0
counts = 0
for i in range(len(test_output["document"])):
    temp = []
    for j in range(len(test_output["document"][i])):
        if int(yo[count-1]) == 1 and answers[count-1][1] > 0.99:
            temp.append(test_output["document"][i][j])
        count += 1
    pers_output["predicted_summary"][i] = temp

# c = 0

# for i in range(len(test_output["document"])):
#     temp = []
#     index = list(range(c, c+len(test_output["document"][i])))
#     try:
#         index = np.sort(np.argpartition(answers[index][:, 1], -4)[-4:])
#     except:
#         index = np.sort(index)
#     for j in range(len(test_output["document"][i])):
#         if j in index:
#             temp.append(test_output["document"][i][j])
#     test_output["predicted_summary"][i] = temp
#     c += len(test_output["document"][i])
    
    
for i in range(len(pers_output["predicted_summary"])):
    pers_output["predicted_summary"][i] = ". ".join(pers_output["predicted_summary"][i])

In [272]:
test_output.summary[2414]

"Ronaldo bagged five goals in 9 - 1 victory against Granada Benzema scored twice for Carlo Ancelotti 's Real Madrid side Benzema *believes* Ronaldo ' deserves everything he has achieved '"

In [273]:
test_output.predicted_summary[2414]

"it was the first time in his career that Ronaldo has scored five goals in a single match and saw him become only the third player to reach the 300 - goal mark for Real Madrid , a tally achieved in only 287 appearances. the Frenchman added : ' Him ( Ronaldo ) , Gareth Bale and me , we 're in good form , like always. ' Real Madrid 's performance was a timely one , coming after they had lost three of their previous five matches in La Liga and the Champions League - including a 2 - 1 loss to bitter rivals Barcelona in their last outing. sunday 's win saw Real Madrid close the gap to Barcelona back to one point ahead of the league leaders ' clash at Celta Vigo on sunday evening , and coach Carlo Ancelotti is looking to the future with optimism"

In [277]:
pers_output.predicted_summary[2414]

"Benzema hailed team - mate Ronaldo as a ' phenomenon ' after the Portuguese struck five goals in Real Madrid 's 9 - 1 mauling of Granada. it was the first time in his career that Ronaldo has scored five goals in a single match and saw him become only the third player to reach the 300 - goal mark for Real Madrid , a tally achieved in only 287 appearances. on sunday , Benzema said on his club 's website : ' Ronaldo is a phenomenon , he is always looking for goals and ways to help the team. he deserves everything he has achieved. ' Ronaldo ( centre ) scored five goals during Real Madrid 's 9 - 1 La Liga victory against Granada Benzema also scored twice after Gareth Bale had opening the scoring as Real Madrid 's much - vaunted ' BBC ' strikeforce combined to score eight of their side 's nine goals , with the other coming from a Diego Mainz own goal. the Frenchman added : ' Him ( Ronaldo ) , Gareth Bale and me , we 're in good form , like always. we 're helping the team and sometimes we sc

In [276]:
test_output.document[2414]

["Benzema hailed team - mate Ronaldo as a ' phenomenon ' after the Portuguese struck five goals in Real Madrid 's 9 - 1 mauling of Granada",
 'it was the first time in his career that Ronaldo has scored five goals in a single match and saw him become only the third player to reach the 300 - goal mark for Real Madrid , a tally achieved in only 287 appearances',
 "on sunday , Benzema said on his club 's website : ' Ronaldo is a phenomenon , he is always looking for goals and ways to help the team",
 'he deserves everything he has achieved',
 "' Ronaldo ( centre ) scored five goals during Real Madrid 's 9 - 1 La Liga victory against Granada Benzema also scored twice after Gareth Bale had opening the scoring as Real Madrid 's much - vaunted ' BBC ' strikeforce combined to score eight of their side 's nine goals , with the other coming from a Diego Mainz own goal",
 "the Frenchman added : ' Him ( Ronaldo ) , Gareth Bale and me , we 're in good form , like always",
 "we 're helping the team 